In [ ]:
import ee
import geemap
import sys
import os
import geopandas as gpd

In [ ]:

# =========================================================================
# 0. DEFINIÇÕES DE CAMINHO LOCAL E CONFIGURAÇÃO
# =========================================================================

# Variáveis Locais
OUTPUT_FOLDER_MAIN = r'D:\\POS\\!FW_HAND_Osmnx\\!PROJETO_FINAL_4_modulos'
ROI_LOCAL_PATH = r'inundacao.geojson'
OUTPUT_FOLDER_BUILDINGS = os.path.join(OUTPUT_FOLDER_MAIN, 'poen3Building')

# Variáveis do Google Earth Engine
OPEN_BUILDINGS_ID = "GOOGLE/Research/open-buildings/v3/polygons"

# Regra para criar as pastas (Mantida, embora não vá fazer download)
try:
    os.makedirs(OUTPUT_FOLDER_BUILDINGS, exist_ok=True)
    print(f"Pasta local de destino verificada/criada em: {OUTPUT_FOLDER_BUILDINGS}")
except OSError as e:
    print(f"Erro ao criar a pasta local: {e}")
    # Não sair aqui, apenas exibir o erro, pois o principal é a visualização
    # sys.exit(1)


In [ ]:
# =========================================================================
# 1. INICIALIZAÇÃO DO EARTH ENGINE E MAPA GEEMAP
# =========================================================================
try:
    ee.Initialize()
    print("Google Earth Engine inicializado com sucesso.")
except Exception as e:
    print(f"Erro ao inicializar o GEE. Verifique sua autenticação: {e}")
    sys.exit(1)

# ➡️ INICIALIZAÇÃO DO MAPA INTERATIVO (geemap)
Map = geemap.Map()
print("Mapa geemap inicializado.")

In [ ]:
# =========================================================================
# 2. CARREGAMENTO E PROCESSAMENTO DOS DADOS
# =========================================================================
print("\nIniciando o processamento de dados...")

try:
    # Carregar a área de interesse (ROI)
    print(f"Carregando ROI do arquivo local: {ROI_LOCAL_PATH}")
    gdf_roi = gpd.read_file(ROI_LOCAL_PATH)

    # Converter a geometria para um ee.FeatureCollection
    roi_geojson = gdf_roi.geometry.__geo_interface__
    ee_features = [ee.Feature(ee.Geometry(feature['geometry'])) for feature in roi_geojson['features']]
    roi_feature_collection = ee.FeatureCollection(ee_features)
    roi_geometry = roi_feature_collection.geometry()
    print("ROI carregada e convertida para geometria do GEE.")

    # Carregar e filtrar o Dataset Open Buildings
    print("Filtrando construções do Open Buildings...")
    buildings = ee.FeatureCollection(OPEN_BUILDINGS_ID)
    buildings_filtrados = buildings.filterBounds(roi_geometry)
    print(f"Número de construções filtradas (estimado): {buildings_filtrados.size().getInfo()}")

except Exception as e:
    print(f"Erro no processamento dos dados: {e}")
    sys.exit(1)

In [ ]:
# =========================================================================
# 3. ESTILIZAÇÃO E VISUALIZAÇÃO COM GEEMAP
# =========================================================================
print("\nAplicando estilo e adicionando camadas ao mapa...")

# 3.1. Estilização das Construções (Simulando FeatureView com rules)
# Usaremos featureCollection.style() para aplicar estilos baseados na confiança
def style_buildings(feature):
    """Função para aplicar estilo (cor) à feição com base na propriedade 'confidence'."""

    # ➡️ SEU PADRÃO DE CORES:
    # confidence >= 0.75: '00FF00' (Verde)
    # confidence >= 0.7 && confidence < 0.75: 'FFFF00' (Amarelo)
    # confidence >= 0.65 && confidence < 0.7: 'FF0000' (Vermelho)

    confidence = ee.Number(feature.get('confidence'))

    # Condições para a cor
    color = ee.Algorithms.If(
        confidence.gte(0.75), '00FF00',  # Verde
        ee.Algorithms.If(
            confidence.gte(0.7).And(confidence.lt(0.75)), 'FFFF00',  # Amarelo
            ee.Algorithms.If(
                confidence.gte(0.65).And(confidence.lt(0.7)), 'FF0000',  # Vermelho
                'FFFFFF' # Cor padrão/outros (opcional)
            )
        )
    )

    # Aplicar o estilo, usando 'color' apenas para a 'outline' (borda)
    # O FeatureCollection precisa ser convertido para uma FeatureCollection estilizada
    return feature.set({
        'style': {
            'color': color, # Cor da borda
            'fillColor': '00000000', # Transparente (sem preenchimento)
            'width': 1 # Espessura da linha
        }
    })

# Aplicar o mapeamento de estilo
buildings_estilizados = buildings_filtrados.map(style_buildings)

# Definição de parâmetros de visualização para o layer de FeatureCollection estilizada
# 'pointSize' e 'styleProperty' são cruciais para usar os estilos definidos acima
vis_params_buildings = {
    'pointSize': 3,
    'styleProperty': 'style'
}

# 3.2. Estilização da ROI (Borda sem Preenchimento)
# Estilo de borda vermelha e sem preenchimento
vis_params_roi = {
    'color': 'FF0000',  # Cor da borda
    'fillColor': '00000000' # Transparente
}

# 3.3. Adicionar Camadas ao Mapa
Map.addLayer(roi_feature_collection.style(**vis_params_roi), {}, 'ROI (Borda)')
Map.addLayer(buildings_estilizados, vis_params_buildings, 'Construções Open Buildings (Confiança)')

# 3.4. Centralizar e Exibir o Mapa
Map.centerObject(roi_geometry, 14) # Centraliza na ROI com um bom zoom

# Exibir o mapa interativo
# Se você estiver em um ambiente Jupyter Notebook/JupyterLab/Colab, isso exibirá o mapa.
display(Map)
print("\nMapa exibido com as construções estilizadas pela confiança (somente borda).")